In [ ]:
import pandas as pd
import numpy as np

In [ ]:
key = #add your OpenAI key

In [ ]:
#load unlabeled test data
test_data = ?

In [ ]:
reviews = list(test_data['Review'])
index_list = #list of corresponding index

In [ ]:
import openai
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=key
)


# Define a list of reviews


# Define the aspects you are interested in
aspects = ['seat', 'crew flight',
       'food and beverages', 'on ground services', 'entertainment', 'cabin comfort',
       'boarding & disembarking', 'attendance in case of problems', 'punctuality',
       'cancellation', 'onboard service', 'customer service']

# Define the function to analyze sentiment for each aspect
def analyze_aspects(review, aspects, index):
    prompt = f"Analyze the following review and provide sentiment ('Positive', 'Neutral', 'Negative', 'not mentioned') for each aspect: {', '.join(aspects)}. Review: {review}. Be careful before stating 'not mentioned'."
    
    response = client.chat.completions.create(
        model="gpt-4",  # or "gpt-4" if you have access to GPT-4
        messages=[
            {"role": "system", "content": "Concise and precise"},
            {"role": "user", "content": prompt},
        ],
        max_tokens=150,
        temperature=1,
        n=1
    )
    # Extract the text from the response
    sentiment_analysis = response.choices[0].message.content.strip()
    
    # Parse the sentiment analysis into a dictionary
    sentiment_dict = {}
    for line in sentiment_analysis.split('\n'):
        if ': ' in line:  # Check if the line contains the ': ' delimiter
            aspect, sentiment = line.split(': ', 1)  # Use maxsplit=1 to avoid splitting more than once
            sentiment_dict[aspect.strip()] = sentiment.strip()
        else:
            # Handle the case where the line doesn't have the expected format
            print(f"Unexpected line format: {line}")
    
    # If sentiment_dict is empty, return an empty DataFrame with the aspect names as columns
    if not sentiment_dict:
        df = pd.DataFrame(columns=aspects)
    else:
        # Convert the dictionary to a DataFrame
        df = pd.DataFrame(sentiment_dict, index=[0])

    df['Index'] = index
    # Replace 'Not mentioned' with NaN
    df.replace('not mentioned', np.nan, inplace=True)
    
    return df


# Create an empty list to store each DataFrame
df_list = []

# Analyze each review and store the resulting DataFrame
for index, review in zip(index_list, reviews):
    sentiment_df = analyze_aspects(review, aspects, index)
    df_list.append(sentiment_df)
    print('review n°:', index)
    print(f"Sentiment Analysis: {sentiment_df}\n")

# Concatenate all DataFrames into one
final_df = pd.concat(df_list, ignore_index=True)
final_df

If cleaning is needed:

In [ ]:
# Function to standardize column names
def standardize_columns(df):
    df.columns = [col.strip().lower().replace(' ', '_').replace('&', 'and').replace('-_','').replace('-','_').replace('/','_') for col in df.columns]
    return df

df_list2 = [standardize_columns(df) for df in df_list]
final_df = pd.concat(df_list2, ignore_index=True)
final_df

If GPT outputs are not all one the defined categories: get all unique categories values and ask (web) ChatGPT for a mapping dict as below.

In [ ]:
# Function to get unique values for each column
def get_unique_values(df):
    unique_values = {}
    for column in df.columns:
        # Collecting unique values from each column
        unique_values[column] = df[column].unique().tolist()
    return unique_values

# Applying the function to your DataFrame
unique_values = get_unique_values(final_df)
unique_values

In [ ]:
# Mapping dictionary
mapping_dict = {
    'Neutral': 'Neutral',
    'Neutral,': 'Neutral',
    'Neutral (due to communication problems)': 'Neutral',
    'Positive': 'Positive',
    'Positive,': 'Positive',
    'Positive.': 'Positive',
    'Positive (based on pleasant personnel)': 'Positive',
    'Negative': 'Negative',
    'Negative,': 'Negative',
    'Negative.': 'Negative',
    'Negative (Passenger complained about being booked into Economy Coach at the same price they paid for First Class)': 'Negative',
    'Negative (Passenger encountered rudeness and inconsistency from the crew)': 'Negative',
    "Negative (as they don't know Russian)": 'Negative',
    'Negative (Not being attended to upon the San Francisco arrival plus an issue with ticketing agents)': 'Negative',
    'Negative (Seat problem)': 'Negative',
    'Negative (Departure from Tahiti was late and the Jetbridge at San Francisco had no attendant)': 'Negative',
    'Negative (Passenger threatened to be removed from flight for raising safety concerns, inadequate compensation acknowledged)': 'Negative',
    'Negative (Significant delay from Tahiti and missing the connecting': 'Negative',
    'Negative (speculation based on negative review of food and beverages and inability of attendants to communicate)': 'Negative',
    'Not mentioned': 'Not mentioned',
    'Not mentioned,': 'Not mentioned',
    'Not Mentioned': 'Not mentioned'
}
# Function to map values
def map_values(val):
    return mapping_dict.get(val, val)  # Return mapped value if it exists, otherwise return the original value

# Apply mapping to all DataFrame values
df_mapped = final_df.applymap(map_values)
#df_mapped.replace('Not mentioned', np.nan, inplace=True)
# Display the transformed DataFrame
print(df_mapped)

save df_mapped of new labeled testing set